In [1]:
import os
print(os.getcwd())

/home/common/komarov.na/my_scripts


In [2]:
import pandas as pd

In [3]:
df1 = pd.read_table('check_flanks_R1.txt', sep='\t')
df1 = df1.drop(['Unnamed: 0', 'ACTIONS'], axis=1, )
print(df1.shape)
df1.head()

(870, 11)


,QUERY,SCORE,START,END,QSIZE,IDENTITY,CHROM,STRAND,START.1,END.1,SPAN
0,1_R1,66,1,66,66,100.0%,chr1_KV880763v1_alt,-,290696,290761,66
1,1_R1,66,1,66,66,100.0%,chr1,-,108278702,108278767,66
2,1_R1,66,1,66,66,100.0%,chr1_KV880763v1_alt,+,427459,427524,66
3,1_R1,66,1,66,66,100.0%,chr1,+,108415614,108415679,66
4,1_R1,25,25,52,66,96.5%,chr3,+,30191242,30191272,31


In [4]:
df2 = pd.read_table('check_flanks_R2.txt', sep='\t')
df2 = df2.drop(['Unnamed: 0', 'ACTIONS'], axis=1, )
print(df2.shape)
df2.head()

(1030, 11)


,QUERY,SCORE,START,END,QSIZE,IDENTITY,CHROM,STRAND,START.1,END.1,SPAN
0,1_R2,71,1,71,71,100.0%,chr1_KV880763v1_alt,-,290696,290766,71
1,1_R2,71,1,71,71,100.0%,chr1,-,108278702,108278772,71
2,1_R2,71,1,71,71,100.0%,chr1_KV880763v1_alt,+,427454,427524,71
3,1_R2,71,1,71,71,100.0%,chr1,+,108415609,108415679,71
4,1_R2,34,16,71,71,94.9%,chr2,+,229496567,229496651,85


In [64]:
df1_1 = df1[df1['QUERY'] == '5_R1'].copy(deep=True)
df1_1.shape[0]

15

In [65]:
df2_1 = df2[df2['QUERY'] == '5_R2'].copy(deep=True)
df2_1.shape[0]

64

In [66]:
tmp_2darray = []
for i, row_1 in df1_1.iterrows():
    
    tmp_df = df2_1[df2_1['CHROM'] == row_1['CHROM']]
    for i, row_tmp in tmp_df.iterrows():
        
        if abs(row_tmp['START.1'] - row_1['START.1']) <= 1200 and abs(row_tmp['END.1'] - row_1['END.1']) <= 1200:
            tmp_2darray.append([row_1['QUERY'], row_1['SCORE'], row_1['START'], row_1['END'], row_1['IDENTITY'],
                                row_1['CHROM'], row_1['STRAND'], row_1['START.1'], row_1['END.1'],
                                row_tmp['SCORE'], row_tmp['START'], row_tmp['END'], row_tmp['IDENTITY'], 
                                row_tmp['CHROM'], row_tmp['START.1'], row_tmp['END.1']])
merget_df = pd.DataFrame(tmp_2darray, columns=['QUERY', 'SCORE_1', 'START_1', 'END_1', 'IDENTITY_1', 'CHROM_1', 'STRAND_1', 'START.1_1', 'END.1_1',
                                   'SCORE_2', 'START_2', 'END_2', 'IDENTITY_2', 'CHROM_2', 'START.1_2', 'END.1_2'])
merget_df = merget_df[merget_df['SCORE_1'] > 40]
merget_df = merget_df.drop(merget_df[merget_df['CHROM_1'].str.contains('alt')].index, axis=0)
#merget_df = merget_df[merget_df['SCORE_2'] > 60]
merget_df = merget_df[merget_df['START_1'] < 3]
#merget_df.sort_values('END_1' )
print(merget_df.shape[0])
merget_df

1


,QUERY,SCORE_1,START_1,END_1,IDENTITY_1,CHROM_1,STRAND_1,START.1_1,END.1_1,SCORE_2,START_2,END_2,IDENTITY_2,CHROM_2,START.1_2,END.1_2
0,5_R1,59,1,61,95.0%,chr3,-,81813044,81813103,69,2,72,95.8%,chr3,81813044,81813113


In [67]:
for i, row in merget_df.iterrows():
    low_coord = min(row['START.1_1'], row['START.1_2'], row['END.1_1'], row['END.1_2'])
    high_coord = max(row['START.1_1'], row['START.1_2'], row['END.1_1'], row['END.1_2'])
    print(row['SCORE_1'],row['START_1'],row['END_1'], 
          f"{row['CHROM_1']}:{low_coord}-{high_coord}", 
          row['SCORE_2'],row['START_2'],row['END_2'],)

59 1 61 chr3:81813044-81813113 69 2 72


In [89]:
merget_df[merget_df['CHROM_1']=='chr5']

,QUERY,SCORE_1,START_1,END_1,IDENTITY_1,CHROM_1,START.1_1,END.1_1,SCORE_2,START_2,END_2,IDENTITY_2,CHROM_2,START.1_2,END.1_2


In [16]:
def hamming_dist(x1, x2):
    j = 0
    for i in range(len(x1)):
        if x1[i] != x2[i]:
            j = j + 1
    return j

In [21]:
def find_overlap(str1, str2):
    if len(str1) < len(str2):
        str_tmp = str1
        str1 = str2
        str2 = str_tmp
    if len(str1) >= len(str2):
        delta = len(str1) - len(str2)
        for i in range(len(str1)):
            if delta - i >= 0:
                part1 = str1[i:len(str2) + i]
                part2 = str2
            else:
                part1 = str1[i:]
                part2 = str2[0:len(str2) + delta - i]
            # print(part1, part2)
            if hamming_dist(part1, part2) == 0:
                #print(part1, len(part1))
                return part1
        return 'no overlap'

In [18]:
R1_orig_list = ['GACTCTTCTTACTTTTAAGGACACATATAATTGAAACTATTTTGGAAAACAAGAGAGCAGGGATAG',
 'TACTATAAACACTTCTATGCCAATAAACTAGAAAATCCAGAAGAAATGGATAAATTCCTGGACACATACACCCTCCCAAGAC',
 'TCTAATCCATTTTGGAAGGTTAGTCTTACATGGTTGTTGCATTCCACAACTATCGGTAGATTATCTTTAAAAAGGGAAGCAA',
 'TGGTTTTTTTTTTTTTTTTGACAGTCTTGCTGTTCCCAGGCTGGAGTGCAGTGGTGCGACCTCAG',
 'ATTTTTTTATTTTTTTTAGCCTTGCATTTGCCCGTGAGAGGAGCAGCCACACTGGAACGAG',
 'ATGAAATTTTAAAATTTTTTTCTCTGCCTAGTTGCAGACGAGGAAGTTAGTGATGAGGCACAAGAAAGATTTCAGGCAAAAC',
 'AGTTTTTTTTTTTTTTTTTGAGAGGGAGTCTCGCTCTGTCACCCAGTCTGGAGTGCAGTGGCACAATCTCAG',
 'TTTTCTTCTTTTTTAAGTCTGAATAATATTCTCTTGTTTGTATTCACCACATTTTATGTATCCATTCATCCATCAATAGACA',
 'AGAGTTAAACTCAGGCCCACTCAACTTTAACCCCCATATTCTTTCCTCTACACTACCCTTTCCACTCCAGCATGCTACAATT']

R2_rev_comp = ['CGGCCGACTCTTCTTACTTTTAAGGACACATATAATTGAAACTATTTTGGAAAACAAGAGAGCAGGGATAG',
 'AGCCTACCAACCAAAAAAAGTCCAGGACCAGACAGATTCACAGATGAATTTTACCAGAGGTACAAAGAGGAG',
 'GTTGCATTCCACAACTATCGGTAGATTATCTTTAAAAAGGGAAGCAATTTCACATTTGTCCAGCAGAGGGAG',
 'CCCGGTTTTTTTTTTTTTTTTTTTTTGACAGTCTTGCTGTTCCCAGGCTGGAGTGCAGTGGTGCGACCTCAG',
 'CGCGCCCGGCCATTTTTTTATTTTTTTTAGCCTTGCATTTGCCCGTGAGAGGAGCAGCCACACTGGAACGAG',
 'GATTTCAGGCAAAACTGCCCTCTTGAAGATGGAGGGGGCCATCTGGCAAAGAATGCAGGCAGCCTCTGCGAG',
 'TTTTTTTTTTTTTTTTTTTGAGAGGGAGTCTCGCTCTGTCACCCAGGCTGGAGTGCAGTGGCACACTCTCAG',
 'TCTTGTTTGTATTCACCACATTTTATGTATCCATTCATCCATCAATAGACACTTGATTGATTCCACTTTTAG',
 'ATTCTGAGCATTTCCTGAGTATCCCTGGGGGGTGCAGAACTGTCCTAAGTACTAAGTGAAGAACAGAAGGAG']

In [22]:
for i in range(len(R1_orig_list)):

    print(f'-----{i+1}-----')
    print(len(R1_orig_list[i]))
    print(len(R2_rev_comp[i]))
    overlap = find_overlap(R1_orig_list[i], R2_rev_comp[i])
    print(overlap, len(overlap))

-----1-----
66
71
GACTCTTCTTACTTTTAAGGACACATATAATTGAAACTATTTTGGAAAACAAGAGAGCAGGGATAG 66
-----2-----
82
72
no overlap 10
-----3-----
82
72
GTTGCATTCCACAACTATCGGTAGATTATCTTTAAAAAGGGAAGCAA 47
-----4-----
65
72
no overlap 10
-----5-----
61
72
ATTTTTTTATTTTTTTTAGCCTTGCATTTGCCCGTGAGAGGAGCAGCCACACTGGAACGAG 61
-----6-----
82
72
GATTTCAGGCAAAAC 15
-----7-----
72
72
no overlap 10
-----8-----
82
72
TCTTGTTTGTATTCACCACATTTTATGTATCCATTCATCCATCAATAGACA 51
-----9-----
82
72
ATT 3


In [ ]:
df1_1 = df1_1[df1_1['SCORE'] > 30]
#merget_df = merget_df[merget_df['SCORE_2'] > 60]
df1_1 = df1_1[df1_1['START'] < 2]
#merget_df.sort_values('END_1' )
print(df1_1.shape[0])
df1_1